In [1]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
cats = ['alt.atheism',
 'comp.sys.ibm.pc.hardware',
 'sci.space',
 'soc.religion.christian',
 'talk.religion.misc']
newsgroups_train = fetch_20newsgroups(subset='train', categories=cats)

***Categorías del dataset***

In [2]:
pprint(list(newsgroups_train.target_names))

['alt.atheism',
 'comp.sys.ibm.pc.hardware',
 'sci.space',
 'soc.religion.christian',
 'talk.religion.misc']


***Cantidad de archivos del dataset***

In [3]:
newsgroups_train.target.shape[0]

2639

***Primeras 30 líneas del mensaje***

In [4]:
indice_cuarto_archivo = 3
cuarto_archivo = newsgroups_train.data[indice_cuarto_archivo]
lineas = cuarto_archivo.split('\n')[:30]
print ''.join(map(str,'\n'.join(lineas)))

From: halat@pooh.bears (Jim Halat)
Subject: Re: After 2000 years, can we say that Christian Morality is
Reply-To: halat@pooh.bears (Jim Halat)
Lines: 34

In article <1qjd3o$nlv@horus.ap.mchp.sni.de>, frank@D012S658.uucp (Frank O'Dwyer) writes:
>Firstly, science has its basis in values, not the other way round.
>So you better explain what objective atoms are, and how we get them
>from subjective values, before we go any further.


Atoms are not objective.  They aren't even real.  What scientists call
an atom is nothing more than a mathematical model that describes 
certain physical, observable properties of our surroundings.  All
of which is subjective.  

What is objective, though, is the approach a scientist 
takes in discussing his model and his observations.  There
is no objective science.  But there is an objective approach
which is subjectively selected by the scientist.  Objective
in this case means a specified, unchanging set of rules that
he and his colleagues use to discuss th

***Tema del mensaje***

In [5]:
print newsgroups_train.target_names[newsgroups_train.target[indice_cuarto_archivo]]

talk.religion.misc


*** Categoría, tema? de los primeros 20 mensajes ***

In [6]:
pprint([newsgroups_train.target_names[newsgroups_train.target[a]] for a in range(0,19)])

['alt.atheism',
 'alt.atheism',
 'sci.space',
 'talk.religion.misc',
 'alt.atheism',
 'soc.religion.christian',
 'comp.sys.ibm.pc.hardware',
 'sci.space',
 'talk.religion.misc',
 'sci.space',
 'sci.space',
 'soc.religion.christian',
 'talk.religion.misc',
 'alt.atheism',
 'sci.space',
 'sci.space',
 'comp.sys.ibm.pc.hardware',
 'talk.religion.misc',
 'alt.atheism']


***Sobre los metadatos***

Los metadatos pueden en infuir en la categorización, en primer lugar hay palabras que se repiten para todos los temas: 'Subject', 'From' y hay otras que no tienen relación directamente con el tema pero podrían influir negativamente: una dirección de email puede escribir habitualmente sobre un tema y esporádicamente sobre otro, sin embargo ésto no debería pesar en la categorización. Lo anterior es un caso de overfitting de los datos.

***Se eliminan los metadatos***

In [7]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
newsgroups_train = fetch_20newsgroups(subset='train', categories=cats,remove=('headers'))

In [8]:
cuarto_archivo = newsgroups_train.data[indice_cuarto_archivo]
lineas = cuarto_archivo.split('\n')[:30]
print ''.join(map(str,'\n'.join(lineas)))

In article <1qjd3o$nlv@horus.ap.mchp.sni.de>, frank@D012S658.uucp (Frank O'Dwyer) writes:
>Firstly, science has its basis in values, not the other way round.
>So you better explain what objective atoms are, and how we get them
>from subjective values, before we go any further.


Atoms are not objective.  They aren't even real.  What scientists call
an atom is nothing more than a mathematical model that describes 
certain physical, observable properties of our surroundings.  All
of which is subjective.  

What is objective, though, is the approach a scientist 
takes in discussing his model and his observations.  There
is no objective science.  But there is an objective approach
which is subjectively selected by the scientist.  Objective
in this case means a specified, unchanging set of rules that
he and his colleagues use to discuss their science.

This is in contrast to your Objective Morality.  There may be an
objective approach to subjectively discuss your beliefs on
morality.  But t

***Modelo vectorial***

La extracción de atributos consiste en transformar datos en general como texto o imágenes en atributos numéricos utilizables en métodos de aprendizaje automático. Cuando se trata de texto resulta imposible introducir una secuencia de simbolos de largo variable en los algoritmos de aprendizaje automático ya que la mayoría de ellos espera vectores de atributos númericos de tamaño fijo. La alternativa consiste en tokenizar los textos de forma de dividirlos en simbolos independientes llamados tokens utilizando los espacios y la puntuación, para posteriormente contar las ocurrencias de dichos tokens y normalizar el resultado. Cada token será considerado un atributo y la representación se denomina **Bag-Of-Words**. Dicha representación no considera el orden original de los tokens en el texto. **Count Vectorizer** es una implementación de extración de atributos de texto que realiza la tokenización y el conteo de las ocurrencias de cada token, **TfidfVectorizer** realiza además la normalización de dicho conteo: sucede que existen cierto tipo de palabras como los artículos que se repiten en la gran mayoría de los textos y que tienen poco significado real en el significado del texto, sin normalización el conteo de éste tipo de palabras podrían restar importancia de otras palabras menos frecuentes pero más significativas en los algoritmos utilizados.

**falta:  qué es tf-idf y cómo se implementa en Python.**

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(newsgroups_train.data)

***Tamaño del vocabulario:***

In [10]:
vectors.shape[1]

36583

***Tamaño maximo de un documento:***

Conclusion: array esparso....

***Categorización***

In [11]:
#Vectorizo el conjunto de test
newsgroups_test = fetch_20newsgroups(subset='test',categories=cats,remove=('headers'))
vectors_test = vectorizer.transform(newsgroups_test.data)

In [16]:
#Metodo Naive Bayes Multinomial
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
clf = MultinomialNB(alpha=.01)
clf.fit(vectors, newsgroups_train.target)
pred = clf.predict(vectors_test)
print metrics.f1_score(newsgroups_test.target, pred, average='weighted')
confusion_matrix(newsgroups_test.target,pred)

0.875028553952


array([[250,   3,   5,  37,  24],
       [  1, 386,   5,   0,   0],
       [  8,   6, 373,   4,   3],
       [ 10,   3,   4, 376,   5],
       [ 37,   4,   9,  45, 156]])

In [14]:
#Metodo stochastic gradient descent
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
clf = SGDClassifier()
clf.fit(vectors, newsgroups_train.target)
pred = clf.predict(vectors_test)
metrics.f1_score(newsgroups_test.target, pred, average='weighted')

0.86399742824387871